In [1]:
from src.utils.bq import BQ
from google.cloud import bigquery

# Initialize BigQuery connection
bq = BQ()

# Get transaction data - FIXED QUERY
TRANSACTION_QUERY = """
WITH base AS (
  SELECT
    customer_id,
    case when market = 'us' then 'US'
    when market = 'uk' then 'UK'
    else 'EU' end as market,
    transaction_completed_datetime
  FROM `mpb-data-science-dev-ab-602d.dsci_daw.STV`
  WHERE transaction_state = 'COMPLETED'
    AND transaction_completed_datetime IS NOT NULL
),
first_two AS (
  SELECT
    customer_id,
    market,
    ARRAY_AGG(transaction_completed_datetime ORDER BY transaction_completed_datetime) AS txns
  FROM base
  GROUP BY customer_id, market
)
SELECT
  customer_id,
  market,
  DATE(txns[OFFSET(0)]) AS first_txn_date,
  CASE 
    WHEN ARRAY_LENGTH(txns) >= 2 THEN
      DATE_DIFF(DATE(txns[OFFSET(1)]), DATE(txns[OFFSET(0)]), DAY)
    ELSE 0
  END AS days_between_first_and_second
FROM first_two;
"""

job_config = bigquery.QueryJobConfig()
dtypes = {
    'customer_id': 'int32',
    'market': 'string',
    'first_txn_date': 'datetime64[ns]',
    'days_between_first_and_second': 'int32'
}

df = bq.to_dataframe(TRANSACTION_QUERY, job_config=job_config, dtypes=dtypes)

/Users/anthonybeech/python/dsci/ltv/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
